In [1]:
from line_solver import *
import numpy as np

In [2]:
# Model creation
# model = JMT2LINE('example_openModel_3.jsimg')  # Alternative: load from JMT file
model = Network('myModel')

print(f"Created model: {model.get_name()}")

Created model: myModel


In [3]:
# Block 1: Network nodes
node = np.empty(5, dtype=object)
node[0] = Source(model, 'Source 1')
node[1] = Queue(model, 'Queue 1', SchedStrategy.PS)
node[2] = ClassSwitch(model, 'ClassSwitch 1')  # Class switching is embedded in the routing matrix P
node[3] = Sink(model, 'Sink 1')
node[4] = Queue(model, 'Queue 2', SchedStrategy.PS)

In [4]:
# Block 2: Job classes  
jobclass = np.empty(3, dtype=object)
jobclass[0] = OpenClass(model, 'Class A', 0)
jobclass[1] = OpenClass(model, 'Class B', 0)
jobclass[2] = OpenClass(model, 'Class C', 0)

In [5]:
# Arrival processes
node[0].set_arrival(jobclass[0], Exp.fit_mean(0.500000))  # (Source 1, Class A)
node[0].set_arrival(jobclass[1], Exp.fit_mean(1.000000))  # (Source 1, Class B)
node[0].set_arrival(jobclass[2], Disabled.getInstance())  # (Source 1, Class C) - no direct arrivals

In [6]:
# Service processes at Queue 1
node[1].set_service(jobclass[0], Exp.fit_mean(0.200000))  # (Queue 1, Class A)
node[1].set_service(jobclass[1], Exp.fit_mean(0.300000))  # (Queue 1, Class B)
node[1].set_service(jobclass[2], Exp.fit_mean(0.333333))  # (Queue 1, Class C)

In [7]:
# Service processes at Queue 2
node[4].set_service(jobclass[0], Exp.fit_mean(1.000000))  # (Queue 2, Class A)
node[4].set_service(jobclass[1], Exp.fit_mean(1.000000))  # (Queue 2, Class B)
node[4].set_service(jobclass[2], Exp.fit_mean(0.150000))  # (Queue 2, Class C)

In [8]:
# Block 3: Network topology and class switching
# Initialize class switching matrix (identity matrix - no class change at the switch node itself)
C = node[2].init_class_switch_matrix()
C = np.eye(len(jobclass))  # Use number of classes for identity matrix size
node[2].set_class_switching_matrix(C)

In [9]:
# Initialize routing matrix
P = model.init_routing_matrix()

# Routing for Class A (index 0)
P.set(jobclass[0], jobclass[0], node[0], node[1], 1.0)  # (Source 1, Class A) -> (Queue 1, Class A)
P.set(jobclass[0], jobclass[0], node[1], node[2], 1.0)  # (Queue 1, Class A) -> (ClassSwitch 1, Class A)
P.set(jobclass[0], jobclass[2], node[2], node[4], 1.0)  # (ClassSwitch 1, Class A) -> (Queue 2, Class C) - class switch!
P.set(jobclass[2], jobclass[2], node[4], node[3], 1.0)  # (Queue 2, Class C) -> (Sink 1, Class C)

# Routing for Class B (index 1)
P.set(jobclass[1], jobclass[1], node[0], node[1], 1.0)  # (Source 1, Class B) -> (Queue 1, Class B)
P.set(jobclass[1], jobclass[1], node[1], node[2], 1.0)  # (Queue 1, Class B) -> (ClassSwitch 1, Class B)
P.set(jobclass[1], jobclass[2], node[2], node[4], 1.0)  # (ClassSwitch 1, Class B) -> (Queue 2, Class C) - class switch!
P.set(jobclass[2], jobclass[2], node[4], node[3], 1.0)  # (Queue 2, Class C) -> (Sink 1, Class C)

# Routing for Class C (index 2) - no arrivals, but define internal routing
P.set(jobclass[2], jobclass[2], node[0], node[1], 1.0)  # (Source 1, Class C) -> (Queue 1, Class C) - not used
P.set(jobclass[2], jobclass[2], node[1], node[2], 1.0)  # (Queue 1, Class C) -> (ClassSwitch 1, Class C) - not used
P.set(jobclass[2], jobclass[2], node[2], node[4], 1.0)  # (ClassSwitch 1, Class C) -> (Queue 2, Class C)
P.set(jobclass[2], jobclass[2], node[4], node[3], 1.0)  # (Queue 2, Class C) -> (Sink 1, Class C)

model.link(P)

In [10]:
# Solver options
options = Solver.default_options()
options['keep'] = True
options['verbose'] = 1
options['cutoff'] = np.array([[1,1,0],[3,3,0],[0,0,3]])  # 5 nodes x 3 classes cutoff matrix
options['seed'] = 23000
options['samples'] = int(1e5)

In [11]:
# Aligned with JAR test scenarios for oqn_cs_routing
# JAR tests: CTMC(cutoff=[[1,1,0],[3,3,0],[0,0,3]]), MVA(), MAM(), NC(), Fluid(), 
#           JMT(seed=23000, samples=100000), SSA(seed=23000, samples=100000, @Disabled)

solver = np.array([], dtype=object)

# CTMC with specific cutoff matrix (matches JAR)  
solver = np.append(solver, CTMC(model, cutoff=np.array([[1,1,0],[3,3,0],[0,0,3]])))

# MVA with default settings (matches JAR)
solver = np.append(solver, MVA(model))

# MAM with default settings (matches JAR)
solver = np.append(solver, MAM(model))

# NC with default settings (matches JAR)
solver = np.append(solver, NC(model))

# Fluid with default settings (matches JAR)
solver = np.append(solver, FLD(model))

# JMT with seed=23000, samples=100000 (matches JAR)
# Commented out: Python native JMT handler doesn't support ClassSwitch nodes
# solver = np.append(solver, JMT(model, seed=23000, samples=100000))

# DES with seed=23000, samples=100000 (matches MATLAB)
solver = np.append(solver, DES(model, seed=23000, samples=100000))

# SSA with seed=23000, samples=100000 (matches JAR, but disabled in JAR tests)
# Commented out per JAR @Disabled annotation
# solver = np.append(solver, SSA(model, seed=23000, samples=100000))

In [12]:
# Execute all solvers and collect results
AvgTable = np.empty(len(solver), dtype=object)

for s in range(len(solver)):
    AvgTable[s] = solver[s].avg_table()

CTMC solver using state space cutoff = [[1 1 0]
 [3 3 0]
 [0 0 3]] for open/mixed model.
      Station JobClass   QLen   Util  RespT  ResidT   ArvR   Tput
     Source 1  Class A 0.0000 0.0000 0.0000  0.0000 2.0000 2.0000
     Source 1  Class B 0.0000 0.0000 0.0000  0.0000 1.0000 1.0000
      Queue 1  Class A 0.8844 0.3832 0.4616  0.4422 1.9158 1.9158
      Queue 1  Class B 0.7109 0.2970 0.7180  0.7109 0.9901 0.9901
ClassSwitch 1  Class C 0.6183 0.4136 0.2243  0.2243 2.7571 2.7571
 Station JobClass   QLen  Util  RespT  ResidT  ArvR  Tput
Source 1  Class A 0.0000  0.00 0.0000  0.0000   0.0   2.0
 Queue 1  Class A 0.6667  0.40 0.3333  0.3333   2.0   2.0
 Queue 1  Class B 0.0000  0.00 0.0000  0.0000   1.0   1.0
 Queue 1  Class C 0.0000  0.00 0.0000  0.0000   1.0   1.0
 Queue 2  Class A 0.0000  0.00 0.0000  0.0000   2.0   2.0
 Queue 2  Class B 0.0000  0.00 0.0000  0.0000   1.0   1.0
 Queue 2  Class C 0.1765  0.15 0.1765  0.1765   1.0   1.0
 Station JobClass   QLen   Util  RespT  ResidT  Arv

/home/gcasale/Dropbox/code/line-dev.git/python/line_solver/api/solvers/ctmc/handler.py:834: UserWarning: State space truncation may cause inaccurate results. Consider varying cutoff to assess sensitivity.
  warnings.warn(


 Station JobClass   QLen   Util  RespT  ResidT   ArvR   Tput
Source 1  Class A 0.0000 0.0000 0.0000  0.0000 0.0000 2.0000
Source 1  Class B 0.0000 0.0000 0.0000  0.0000 0.0000 1.0000
Source 1  Class C 0.0000 0.0000 0.0000  0.0000 0.0000 0.0000
 Queue 1  Class A 1.5861 0.3976 0.7532  0.7532 1.9970 1.9969
 Queue 1  Class B 0.9034 0.3000 0.8634  0.8634 0.9967 0.9967
 Queue 2  Class C 0.8506 0.4478 0.2743  0.2743 2.9936 2.9936
